# Handwritten name recognition using ANNs

### Made by Iker García and Eritz Yerga
https://github.com/ikergarcia1996/Handwritten-Names-Recognition

***

Before beginning we will define how we want our program to work.

Change any of these parameters if you want the notebook to function in a different way.

In [1]:
# Define if you want to download data from the original database or use the dataset one already provided and preprocessed
# Use:
# 'load': If you want to load the datase from the directory
# 'download': To download data from the database and process the images
dataset_load_method = 'load'

# Define if you want to save the dataset to a file
save_dataset = False

# Define if you want to load the trained classifiers from the directory
load_classifiers = False

# Define if you want to save the trained classifiers to a file
save_classifiers = True

# Define if you want to save classification test output to a file
save_results = True
if (save_results):
    result_output_file = open('result_output.txt','w') 

# Define if you want to print errors and warnings
enable_error_output = False

# 1. Library imports

We import the needed libraries to run our code. All of these libraries are installable through pip3 except cv2 (_see note bellow_).

*NOTE:* To import _cv2_ you might need to install the _python3-opencv_ package. 

In [2]:
import os
os.system('pip install plotly')
os.system('pip install opencv-contrib-python')

0

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import scipy
import sklearn
import pandas as pd
from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import binarize
from sklearn.neural_network import BernoulliRBM, MLPClassifier
from sklearn.datasets import fetch_mldata
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skimage import data, color, exposure, measure
from skimage.transform import resize
from skimage.feature import hog
from sklearn.manifold import TSNE
from sklearn.externals import joblib
import urllib
from io import StringIO
import cv2
from scipy import ndimage
from difflib import SequenceMatcher
from sys import stdout
from IPython.display import clear_output

ImportError: No module named 'sklearn'

We will also define a small function for displaying purposes:

In [ ]:
def print_percentage(prct, msg=None):
    if (prct > 100 or prct < 0):
        return
    clear_output(wait=True)
    if (msg == None):
        stdout.write("Progress: [")
    else:
        stdout.write(msg+" [")
    end = int(int(prct)/10)
    for i in range(0, end):
        stdout.write("=")
    for i in range(end, 10):
        stdout.write(" ")
    stdout.write("] "+str(prct)+"%")
    stdout.flush()

# 2. Importing the dataset with the urls and names

We import the "Transcriptions of handwriten names" database obtained from https://www.crowdflower.com/data-for-everyone/ . As the csv file in the website has some errors, we fixed those errors using a small program written in Java (see java file included in source: database_fix.java).

In [ ]:
df=pd.read_csv('first_and_last_names_fix.csv', sep=',',header=None)

***

### Functions to create the full dataset and labels

As the database _df_ we imported doesn't contain the images but link to the corresponding images, we need to download them and create a dataset with those and the labels.

The *_delborders_* function eliminates the bottom border noise in case of "first_b" and "last_b" types in the dataset.

In [ ]:
# Here we are applying filters and making it clean.

def delborders(crop):
    cropf = ndimage.gaussian_filter(crop, 0.5)
    cropbin = (cropf<0.8)
    labeled, nr_objects = ndimage.label(cropbin)
    labels_to_delete = []
    for i in range(0, labeled.shape[1]):
        if (labeled[labeled.shape[0]-1][i] > 0):
            labels_to_delete.append(labeled[labeled.shape[0]-1][i])
    
    label_in_delete = False
    for x in range(0, labeled.shape[1]):
        for y in range(0, labeled.shape[0]):
            label_in_delete = False
            for l in range(0, len(labels_to_delete)):
                if (labeled[y][x] == labels_to_delete[l]):
                    label_in_delete = True
            
            if(label_in_delete):
                crop[y][x] = 1.0
    
    return crop

The *_getcrop_* function obtains an image from an entry of the database and returns the image in grayscale and cropped to show just the name part.

In [ ]:
def getcrop(n):
    try: 
        urllib.request.urlretrieve(df[1][n], "temp.jpg")
    except urllib.error.URLError as e:
        return None, False
    img = cv2.imread('temp.jpg')
    imgh, imgw = img.shape[:-1]
    img_rgb = img.copy()
    template = cv2.imread('template.png')
    h, w = template.shape[:-1]

    template_match_success = False
    res = cv2.matchTemplate(img_rgb, template, cv2.TM_CCOEFF_NORMED)
    threshold = .7
    loc = np.where(res >= threshold)
    for pt in zip(*loc[::-1]):
        print(pt)# Switch collumns and rows
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
        croph1 = pt[1]
        croph2 = pt[1]+h
        cropw = pt[0] + w
        template_match_success = True

    if (not template_match_success):
        #Template matching has failed so return...
        return img, False

    if (df[3][n] == 'first' or df[3][n] == 'last'):
        crop = img.copy()[max(croph1-6, 0):min(croph2+6, imgh), cropw:imgw]
    else:
        crop = img.copy()[max(min(croph2+4, imgh-1), 0):imgh, :]
        
    crop = color.rgb2gray(crop)
    if (df[3][n] == 'first_b' or df[3][n] == 'last_b'):
        crop = delborders(crop)
    return crop, True

Next we define the gen_dataset function, this function creates a binarized image list "_data_" and a label list "_labels_" from the first _n_ entries of the database (if not specified uses all the entries of the database) using the crops of the _getcrop_ function and its corresponding label on that index.

In [ ]:
def gen_dataset(n=df.shape[0]):
    data = []
    labels = []
    for i in range(1, n):
        crop, success = getcrop(i)
        if (success):
            data.append(crop)
            labels.append(df[2][i])
        else:
            if (enable_error_output):
                print("[WARNING] Template matching has failed for image: "+str(i))
        print_percentage((i*100/(n-1)), "Fetched "+str(i)+" images:")
    
    print_percentage(100, "Fetched "+str(n-1)+" images:")
    print("")
    print("Finished!")
    return data, labels

### Obtaining the dataset

Now we can download and create the whole dataset and its corresponding label list or load the dataset from the saved files. The process takes a while to complete...

In [ ]:
# Obtain dataset from original data if defined (we can define how much data of the 
# original database we want to use to the function or leave it blank to use all the data)
if (dataset_load_method == 'download'):
    dataset, labels = gen_dataset(10000)

# Load dataset from files
if (dataset_load_method == 'load'):
    dataset = np.load("HandwrittenNames_data.npz")['data']
    labels = np.load("HandwrittenNames_labels.npz")['data']

If specified, the generated dataset can be saved to .npz files using these functions.

In [3]:
# Save dataset to a file if defined
if (save_dataset):
    np.savez("HandwrittenNames_data.npz", data=dataset)
    np.savez("HandwrittenNames_labels.npz", data=labels)

We can plot some images and print their corresponding labels to check that everything is correct: 

In [ ]:
# Change selection to plot a different image and label
selection = 0
plt.imshow(dataset[selection], cmap='gray')
plt.show()
print(labels[selection])
print(str(type(labels[0])))
print(type(dataset[selection]))

# 3. Defining extra helpful functions

In this section of the notebook we will define some functions that will be useful later on.

### get_labels

This function labels the connected components in an image by binarizing it and running a clustering method, it returns the labels and the number of labels it detects.

In [ ]:
def get_labels(crop):
    img = crop.copy() # gray-scale image

    # You could smooth the image (to remove small objects) but we saw better results without using it...
    # blur_radius = 0.5
    # imgf = ndimage.gaussian_filter(img, blur_radius)

    threshold = 0.8

    # Find connected components
    labeled, nr_objects = ndimage.label(img<threshold) 
    #print("Number of objects is " +str(nr_objects))

    return labeled, nr_objects

### get_bboxes

This function gets the bounding boxes to cut each character correctly given the labels obtained from get_labels. It returns a list of each character's bounding boxes (2 2D points).

In [ ]:
def get_bboxes(labeled, nr_objects):
    bboxes = np.zeros((nr_objects, 2, 2), dtype='int')

    x1, y1, x2, y2 = 0, labeled.shape[0], 0, 0
    coord = 0
    cont = 0
    ytop, ybot = 0, 0
    nzero, firstb = False, False

    for x in range(0, labeled.shape[1]):
        nzero, firstb = False, False
        ytop, ybot = 0, 0
        for y in range(0, labeled.shape[0]):
            if (labeled[y][x] > 0):
                nzero = True
                if (not firstb):
                    ytop = y
                    firstb = True
                ybot = y

        if (nzero):
            if (ytop < y1):
                y1 = ytop
            if (ybot > y2):
                y2 = ybot
            if (coord == 0):
                x1 = x
                coord = 1
            elif (coord == 1):
                x2 = x
        elif ((not nzero) and (coord == 1)):
            bboxes[cont][0] = [x1, y1]
            bboxes[cont][1] = [x2, y2]
            cont += 1
            coord = 0
            x1, y1, x2, y2 = 0, labeled.shape[0], 0, 0

    bboxes = bboxes[0:cont]
    return bboxes, cont

### crop_characters

Given an image and character bounding boxes this function crops each character in an image and returns each character's corresponding binarized image in a list.

In [ ]:
def crop_characters(img, bboxes, n):
    characters = []
    for i in range(0, n):
        c = img.copy()[bboxes[i][0][1]:bboxes[i][1][1], bboxes[i][0][0]:bboxes[i][1][0]]
        if (c.shape[0] != 0 and c.shape[1] != 0):
            c = resize(c, (28, 28), mode='constant', cval=1.0, clip=True)
            characters.append((c<0.80).reshape(784))
    return characters, len(characters)

### labelsep

Separates a full name label into a character list. Useful for the training part to have the labels of each character.

In [ ]:
def labelsep(label):
    if (type(label) is str or type(label) is np.str_):
        decomposed_label = list(label)
        labels = []
        for i in range(0, len(decomposed_label)):
            if (decomposed_label[i] != ' '):
                labels.append(decomposed_label[i])
        return labels
    else:
        return []

### get_characters

Given an image from the dataset and its label this function splits each character into one image and a label. The img_only variant doesn't return the labels (useful when testing with full names to save some memory and time).

In [ ]:
def get_characters(image, label):
    labeled, nr_objects = get_labels(image)
    bboxes, n = get_bboxes(labeled, nr_objects)
    characters, n_chars = crop_characters(image, bboxes, n)
    labels = labelsep(label)
    return characters, labels[0:n_chars]

def get_characters_img_only(image):
    labeled, nr_objects = get_labels(image)
    bboxes, n = get_bboxes(labeled, nr_objects)
    characters, n_chars = crop_characters(image, bboxes, n)
    return characters

## Checking results

We can check if it extracts correctly all the data with these plots and prints

In [ ]:
selection = 1
plt.imshow(dataset[selection], cmap='gray')
plt.show()
print(labels[selection])

In [ ]:
characters, charlabels = get_characters(dataset[selection], labels[selection])

In [ ]:
for i in range(0, len(characters)):
    plt.imshow(characters[i].reshape(28,28), cmap='gray')
    plt.show()
    
print(str(charlabels))

# 4. Defining our models

We will define our four classifiers: MLP with RBM features, MLP with HOG features, MLP with PCA features and MLP only.

In [ ]:
# Define if we want to print status of the training process
verbose_classifiers = True

### MLP with PCA features
We define a MLP with 3 layers of 300, 400 and 150 neurons respectively to train it with the PCA features later on.

In [ ]:
# MLP Classifier with PCA (Principal Component Analisis) features
mlp_classifier_PCA = MLPClassifier(hidden_layer_sizes=(300,400,150), max_iter=5000, tol=0.0001, random_state=1, verbose=verbose_classifiers)

# 5. Preparing the train and test batches

### Splitting the dataset into train and test batches

We divide the dataset into 80% train 20% test batches:

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset, labels, test_size=0.2, random_state=0)

In [ ]:
print("Train batch lenghts:")
print("X_train size: "+str(len(X_train)))
print("Y_train size: "+str(len(Y_train)))
print("")
print("Test batch lenghts:")
print("X_test size: "+str(len(X_test)))
print("Y_test size: "+str(len(Y_test)))

### Processing the train and test batches for characters

We process the train and test batches to divide the batches into single character batches instead of full names, this is needed to train and will be useful in case of the test to test per character precision later on.

In [ ]:
X_train_chars = []
Y_train_chars = []
Train_with_inconsistencies = []
z = 0
for i in range(0, len(X_train)):
    print_percentage(i*100/len(X_train), "Processing train image "+ str(i)+" :")
    characters, charlabels = get_characters(X_train[i], Y_train[i])
    if (len(characters) != len(charlabels) or len(characters) == 0 or len(charlabels) == 0):
        if (enable_error_output):
            print("[Warning] Input number "+str(i)+" inconsistent! Skipping this one...")
        Train_with_inconsistencies.append(i)
        z += 1
    else:
        X_train_chars.extend(characters)
        Y_train_chars.extend(charlabels)

print_percentage(100, "Processing train image "+ str(len(X_train))+" :")
print("")
print(str(100-(z*100/len(X_train)))+"% of the data in train batch correctly extracted.")

In [ ]:
print("Character splitted train batch lenghts:")
print("X_train_chars size: "+str(len(X_train_chars)))
print("Y_train_chars size: "+str(len(Y_train_chars)))

In [ ]:
X_test_chars = []
Y_test_chars = []
Test_without_inconsistencies = []
z = 0
for i in range(0, len(X_test)):
    print_percentage(i*100/len(X_test), "Processing test image "+ str(i)+" :")
    characters, charlabels = get_characters(X_test[i], Y_test[i])
    if (len(characters) != len(charlabels) or len(characters) == 0 or len(charlabels) == 0):
        if (enable_error_output):
            print("[Warning] Input number "+str(i)+" inconsistent! Skipping this one...")
        z += 1
    else:
        X_test_chars.extend(characters)
        Y_test_chars.extend(charlabels)
        Test_without_inconsistencies.append(i)

print_percentage(100, "Processing train image "+ str(len(X_test))+" :")
print("")
print(str(100-(z*100/len(X_test)))+"% of the data in test batch correctly extracted.")

In [ ]:
print("Character splitted test batch lenghts:")
print("X_test_chars size: "+str(len(X_test_chars)))
print("Y_test_chars size: "+str(len(Y_test_chars)))

### Transform the data to extract Principal Component Analysis features

We transform the data to extract Principal Component Analysis features for the MLP_PCA classifier.

In [ ]:
# Standardising the values
Scaler = StandardScaler().fit(X_train_chars)
X_pca_train = Scaler.transform(X_train_chars)
# Call the PCA method with 100 components. 
pca = PCA(n_components=100)
pca.fit(X_pca_train)
P_train = pca.transform(X_pca_train)

# Standardising the values
X_pca_test = Scaler.transform(X_test_chars)

# Call the PCA method with 100 components. 
P_test = pca.transform(X_pca_test)

In [ ]:
# We also define a function in case we need to transform something later
def PCA_transform(chars):
    # Standardising the values
    X_pca = Scaler.transform(chars)

    # Call the PCA method with 100 components. 
    P = pca.transform(X_pca)
    return P

We can see how the PCA looks like (only 2 PCA components):

In [ ]:
print_limit = 1000
Target = Y_train_chars[:print_limit]
Target_colors = Y_train_chars[:print_limit]
for i in range(0, len(Target_colors)):
    Target_colors[i] = ord(Target_colors[i])
Characters = go.Scatter(
    x = P_train[:print_limit,0],
    y = P_train[:print_limit,1],
    name = Target,
    hoveron = Target,
    mode = 'markers',
    text = Target,
    showlegend = True,
    marker = dict(
        size = 8,
        color = Target_colors,
        colorscale ='Jet',
        showscale = False,
        line = dict(
            width = 2,
            color = 'rgb(255, 255, 255)'
        ),
        opacity = 0.8
    )
)
data = [Characters]

layout = go.Layout(
    title= 'Principal Component Analysis (PCA)',
    hovermode= 'closest',
    xaxis= dict(
         title= 'First Principal Component',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Second Principal Component',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend= True
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

# 6. Training

### Training/loading classifiers

We feed the train characters and train character labels to each classifier or load the already trained classifiers from files (depending on the selected option):

In [ ]:
if (not load_classifiers):
    rbm_mlp_classifier.fit(X_train_chars[:], Y_train_chars)
else:
    rbm = joblib.load('RBM.pkl')
    mlp = joblib.load('MLP_withRBMfeatures.pkl')
    rbm_mlp_classifier = joblib.load('RBM_MLP_classifier.pkl')

In [ ]:
# We can define if we want to plot the components extracted by the RBM
plot_rbm_features = True

if (plot_rbm_features):
    plt.figure(figsize=(28, 28))
    for i, comp in enumerate(rbm.components_):
        plt.subplot(30, 10, i+1)
        plt.imshow(comp.reshape((28, 28)), cmap=plt.cm.gray_r, interpolation='nearest')
        plt.xticks(())
        plt.yticks(())
    plt.suptitle('300 components extracted by the RBM', fontsize=16)
    plt.subplots_adjust(0.08, 0.02, 0.92, 0.85, 0.08, 0.23)

    plt.show()

In [ ]:
if (not load_classifiers):
    mlp_classifier_PCA.fit(P_train,Y_train_chars)
else:
    mlp_classifier_PCA = joblib.load('MLP_PCA.pkl')

### Saving the classifiers

If enabled, this will save the trained models to files.

In [ ]:
# Save the classifiers if enabled:
if (save_classifiers):
    joblib.dump(rbm, 'RBM.pkl')
    joblib.dump(mlp_classifier_PCA, 'MLP_PCA.pkl')


# 7. Testing the results

### Functions for testing results later

predict_full_name given a full name image and a classifier divides the image into characters and asks the classifier to predict it, afterwards chains the predictions of each characters into a full string.

In [ ]:
# transform should be 'hog' for MLP_HOG classifier and 'pca' for MLP_PCA classifier, otherwise None
def predict_full_name(name, classifier, transform=None):
    characters = get_characters_img_only(name)
    if (transform == 'pca'):
        prediction = classifier.predict(PCA_transform(characters))
    else:
        prediction = classifier.predict(characters)
    strg = ''
    for i in range(0, len(prediction)):
        strg = strg+prediction[i]
    return strg

predict_full_names gets the prediction for each consistent test data and returns the correct ratio and correlation ratio.

In [ ]:
# transform should be 'hog' for MLP_HOG classifier and 'pca' for MLP_PCA classifier, otherwise None
def predict_full_names(classifier, transform=None):
    correlation=0.0
    correct = 0
    for i in range(0,len(Test_without_inconsistencies)):
        predicted_name = predict_full_name(X_test[Test_without_inconsistencies[i]], classifier, transform)
        if (predicted_name == Y_test[Test_without_inconsistencies[i]]):
            correct += 1
        correlation += similar(predicted_name, Y_test[Test_without_inconsistencies[i]])
        print_percentage(i*100/len(X_test),"Making predictions "+str(i)+"/"+str(len(X_test))+":")
    print_percentage(100,"Making predictions "+str(len(X_test))+"/"+str(len(X_test))+":")
    return (correct/len(Test_without_inconsistencies)), (correlation/len(Test_without_inconsistencies))

similar gives a correlation ratio between two strings.

In [ ]:
def similar(a,b):
        return SequenceMatcher(None,a,b).ratio()

### Testing character recognintion

We will test now individual character recognition scores for each classifier using the test batch.

In [ ]:
rbm_mlp_prediction = rbm_mlp_classifier.predict(X_test_chars)
mlp_prediction_PCA = mlp_classifier_PCA.predict(P_test)

In [ ]:
print("MLP classification using RBM features:\n%s\n" % (metrics.classification_report(Y_test_chars, rbm_mlp_prediction)))
print("PCA + MLP classification:\n%s\n" % (metrics.classification_report(Y_test_chars, mlp_prediction_PCA)))

if (save_results):
    result_output_file.write("MLP classification using RBM features:\n%s\n" % (metrics.classification_report(Y_test_chars, rbm_mlp_prediction)))
    result_output_file.write("\nPCA + MLP classification:\n%s\n" % (metrics.classification_report(Y_test_chars, mlp_prediction_PCA)))
    

### Testing full name recognition

We will test now full name recognition scores using the consistent test batch.

In [ ]:
correct_mlp_PCA, corr_mlp_PCA = predict_full_names(mlp_classifier_PCA, 'pca')
correct_mlp, corr_mlp = predict_full_names(mlp_classifier)

In [ ]:
print("Full name test results: ")
print("========================================================================")
print("| Classifier            | Correct percentage      | Correlation ratio  |")
print("========================================================================")
print("| MLP with PCA features | "+str(correct_mlp_PCA)+"     | "+str(corr_mlp_PCA)+" |")
print("========================================================================")

if (save_results):
    result_output_file.write("\n\nFull name test results: ")
    result_output_file.write("\n========================================================================")
    result_output_file.write("\n| Classifier            | Correct percentage      | Correlation ratio  |")
    result_output_file.write("\n========================================================================")
    result_output_file.write("\n| MLP with PCA features | "+str(correct_mlp_PCA)+"     | "+str(corr_mlp_PCA)+" |")
    result_output_file.write("\n========================================================================")

### Visual testing

We can see some predictions visually here by adding indexes to the "indexes" list and for each test data selected in "indexes" we will see the predictions of each classifier.

In [ ]:
indexes = [8, 10, 12, 14, 16]
for ind in indexes:
    mlp_pca_predict = predict_full_name(X_test[ind], mlp_classifier_PCA, 'pca')
    
    print("> Image:")
    plt.imshow(X_test[ind], cmap='gray')
    plt.show()
    print("MLP with PCA features predicted: "+mlp_pca_predict)
    print("")
    
    if (save_results):
        result_output_file.write("\n\n> Real label: "+Y_test[ind])
        result_output_file.write("\nMLP with PCA features predicted: "+mlp_pca_predict)
        result_output_file.write("\n")

In [ ]:
if (save_results):
    result_output_file.close()

## Saving this trained model for further usage.

In [ ]:
from sklearn.externals import joblib
joblib.dump(mlp_classifier_PCA, 'name_recognizer.pkl') 

## Reading from a different process

In [ ]:
names_recog = joblib.load('name_recognizer.pkl')
img = cv2.imread('train1.jpg',0)
img = delborders(img)
print(type(img))
plt.imshow(img,cmap='gray')
plt.show()
img = cv2.imread('train1.jpg',0)
img = delborders(img)
plt.imshow(img,cmap='gray')
plt.show()

In [ ]:
from skimage.filters import try_all_threshold, threshold_isodata
img = cv2.imread('train2.jpg',0)
th = threshold_isodata(img, nbins=256, return_all=False)
image = img > th
plt.imshow(image,cmap='gray')
plt.show()
img2 = cv2.imread('train1.jpg',0)
th2 = threshold_isodata(img2, nbins=256, return_all=False)
image2 = img2 > th2
plt.imshow(image2,cmap='gray')
plt.show()


In [ ]:
# For filtered image
names_recog = joblib.load('name_recognizer.pkl')
mlp_pca_predict = predict_full_name(image, names_recog, 'pca')
print(mlp_pca_predict)

# For filtered image
mlp_pca_predict1 = predict_full_name(image2, names_recog, 'pca')
print(mlp_pca_predict1)